In [1]:
%load_ext autoreload
%autoreload 2
%aimport src

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC

In [2]:
cancer_name = "BLCA"
X, y, _, _, class_mapping = src.data.load_sample_classification_problem(cancer_name)

In [3]:
pipeline = Pipeline([
    ('feature_selection', VarianceThreshold(threshold=0.5)),
    ('classifier', LinearSVC(class_weight="balanced",max_iter=2000))
])

In [4]:
res = src.models.cross_validation(pipeline, X, y, cv=5, n_jobs=2)

In [7]:
res

array([[[4, 0],
        [0, 4]],

       [[3, 1],
        [0, 3]],

       [[4, 0],
        [1, 4]],

       [[3, 1],
        [0, 3]],

       [[3, 0],
        [0, 3]]])